In [1]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SepidehHabibiabad\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SepidehHabibiabad\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\SepidehHabibiabad\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [4]:
pip install feather-format

  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2457 sha256=2f047bad3dee314b9ccf9e0733c2e6344a3b3aacfb1d4967d93528ba06287b5d
  Stored in directory: c:\users\sepidehhabibiabad\appdata\local\pip\cache\wheels\e3\dc\eb\3a9468e9322c8284d7a53eab23e7a4bab6f271fff7a5871e2f
Successfully built feather-format
Note: you may need to restart the kernel to use updated packages.


In [2]:

articles = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/articles.csv')
customers = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/customers.csv')
# sample_submisison = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/sample_submission.csv')
transactions = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/transactions_train.csv')

# Converting the data from CSV to FEATHER file for better perfomance.

In [5]:
#https://medium.com/@rajkrakesh/feather-vs-csv-829472196832
transactions.to_feather('transactions_train.feather')
articles.to_feather('articles.feather')
customers.to_feather('customers.feather')

In [6]:
articles = pd.read_feather('articles.feather')
customers = pd.read_feather('customers.feather')
transactions = pd.read_feather('transactions_train.feather')

# Data Cleaning

In [7]:
# Preprocessing the data for transaction dataset
def customer_id_to_int(x): return int(x[-16:], 16)

transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])
transactions['article_id'] = transactions['article_id'].astype('int32')
transactions['customer_id'] = transactions['customer_id'].apply(customer_id_to_int).astype('int32')
transactions.sales_channel_id = transactions.sales_channel_id.astype('int8')
transactions.price = transactions.price.astype('float32')

In [8]:
# Determining the last month of the purchases
transactions['week'] = 104 - (transactions.t_dat.max() - transactions.t_dat).dt.days // 7
last_month = transactions['week'].max() - 4
transactions = transactions.loc[transactions["week"] >= last_month]
transactions

,t_dat,customer_id,article_id,price,sales_channel_id,week
30488290,2020-08-19,-732324596,784053005,0.050831,2,100
30488291,2020-08-19,-732324596,784053005,0.050831,2,100
30488292,2020-08-19,-732324596,926921001,0.033881,2,100
30488293,2020-08-19,-732324596,868038001,0.050831,2,100
30488294,2020-08-19,-732324596,868038001,0.050831,2,100
...,...,...,...,...,...,...
31788319,2020-09-22,1362182998,929511001,0.059305,2,104
31788320,2020-09-22,1362182998,891322004,0.042356,2,104
31788321,2020-09-22,-324376415,918325001,0.043203,1,104
31788322,2020-09-22,2104975119,833459002,0.006763,1,104


In [9]:
# Method to preprocess the columns with texts in articles dataset, help
# by Bjorn Rexhepi.
ps = nltk.stem.porter.PorterStemmer()
punct = set(string.punctuation)

def preprocess_text(text):
    #text = unidecode.unidecode(text).lower()  # remove accents and lower
    text = ''.join([char for char in text if char not in punct])  # remove punctuation 
    words = nltk.word_tokenize(text) # tokenize
    words = [ps.stem(w) for w in words] # stem words
    text = ' '.join(words) # convert back to text
    return text

In [10]:
for c in articles.columns:
      articles[c] = articles[c].dropna()
articles

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,953450001,953450,5pk regular Placement1,302,Socks,Socks & Tights,1010014,Placement print,9,Black,...,Socks Bin,F,Menswear,3,Menswear,26,Men Underwear,1021,Socks and Tights,Socks in a fine-knit cotton blend with a small...
105538,953763001,953763,SPORT Malaga tank,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,2,H&M+,1005,Jersey Fancy,Loose-fitting sports vest top in ribbed fast-d...
105539,956217002,956217,Cartwheel dress,265,Dress,Garment Full body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,"Short, A-line dress in jersey with a round nec..."
105540,957375001,957375,CLAIRE HAIR CLAW,72,Hair clip,Accessories,1010016,Solid,9,Black,...,Small Accessories,D,Divided,2,Divided,52,Divided Accessories,1019,Accessories,Large plastic hair claw.


In [11]:
# Converting the numbers to integers and preprocessing the texts.
articles['article_id'] = articles['article_id'].astype('int32')
articles['product_code'] = articles['article_id'].astype('int8')
articles['prod_name'] = articles['prod_name'].apply(preprocess_text)
articles['product_group_name'] = pd.factorize(articles['product_group_name'])[0].astype('int8')
articles['product_type_no'] = articles['product_type_no'].astype('int8')
articles['graphical_appearance_no'] = pd.factorize(articles['graphical_appearance_no'])[0].astype('int16')
articles['colour_group_code'] = articles['colour_group_code'].astype('int8')
articles['perceived_colour_value_id'] = articles['perceived_colour_value_id'].astype('int16')
articles['perceived_colour_master_id'] = articles['perceived_colour_master_id'].astype('int16')
articles['department_no'] = articles['department_no'].astype('int8')
articles['index_code'] = pd.factorize(articles['department_no'])[0].astype('int8')
articles['index_group_no'] = articles['index_group_no'].astype('int16')
articles['section_no'] = articles['section_no'].astype('int8')
articles['garment_group_no'] = articles['garment_group_no'].astype('int16')

In [15]:
# Using SBert to encode the information of detail description!
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

In [13]:
pip install sentence_transformers

  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125940 sha256=2f6d738409fc5225f7830195248e2647914b74bb3763fae8ea6b6d1e796a3c0d
  Stored in directory: c:\users\sepidehhabibiabad\appdata\local\pip\cache\wheels\71\67\06\162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [16]:
 # https://www.sbert.net/docs/pretrained_models.html
 sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [17]:
text_features = sbert_model.encode(articles['detail_desc'].map(str).values.tolist()).tolist()
text_transformation = PCA(n_components=16).fit_transform(text_features).tolist()

In [18]:
articles[[f'detail_desc_{i}' for i in range(16)]] = pd.DataFrame(text_transformation)

In [19]:
articles.drop(
        columns=['product_type_name', 'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name',
                 'perceived_colour_master_name', 'department_name', 'index_name', 'index_group_name', 'section_name',
                 'garment_group_name', 'detail_desc'], inplace=True)

In [20]:
articles['prod_name'] = pd.factorize(articles['prod_name'])[0].astype('int8')
articles

,article_id,product_code,prod_name,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,...,detail_desc_6,detail_desc_7,detail_desc_8,detail_desc_9,detail_desc_10,detail_desc_11,detail_desc_12,detail_desc_13,detail_desc_14,detail_desc_15
0,108775015,103,0,-3,0,0,9,4,5,-116,...,0.042811,0.180952,-0.181124,-0.043562,0.117311,-0.006668,-0.177407,-0.005806,0.055835,0.071011
1,108775044,-124,0,-3,0,0,10,3,9,-116,...,0.042811,0.180952,-0.181124,-0.043562,0.117311,-0.006668,-0.177407,-0.005806,0.055835,0.071011
2,108775051,-117,1,-3,0,1,11,1,9,-116,...,0.042811,0.180952,-0.181124,-0.043562,0.117311,-0.006668,-0.177407,-0.005806,0.055835,0.071011
3,110065001,105,2,50,1,0,9,4,5,59,...,0.191096,-0.204776,0.345293,0.212235,0.051536,0.071205,0.018807,0.031909,0.085815,0.052503
4,110065002,106,2,50,1,0,10,3,9,59,...,0.191096,-0.204776,0.345293,0.212235,0.051536,0.071205,0.018807,0.031909,0.085815,0.052503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,953450001,17,-126,46,2,18,9,4,5,20,...,-0.260706,0.062214,0.079748,0.191380,0.003752,0.005283,0.019482,-0.137319,-0.030368,0.174725
105538,953763001,-71,-125,-3,0,0,9,4,5,127,...,0.027513,-0.318021,-0.154747,-0.082080,-0.102836,0.165789,-0.172489,-0.013385,0.119233,0.014247
105539,956217002,-86,-124,9,11,0,9,4,5,105,...,-0.076326,0.151726,-0.171701,0.135284,-0.037159,-0.027511,-0.085334,0.063054,-0.010668,-0.074030
105540,957375001,25,-123,72,4,0,9,4,5,106,...,-0.023462,0.040188,-0.101171,-0.075792,0.074773,-0.009505,-0.009033,0.056453,-0.096665,-0.082185


In [21]:
# https://github.com/radekosmulski/personalized_fashion_recs/blob/main/01_Solution_warmup.ipynb
customers['customer_id'] = customers['customer_id'].apply(customer_id_to_int).astype('int32')
customers['age'].fillna(int((customers['age'].mean())), inplace=True)
for c in ['FN', 'Active']:
    customers[c].fillna(-1, inplace=True)
    customers[c] = customers[c].astype('int8')
customers["fashion_news_frequency"] = pd.factorize(customers["fashion_news_frequency"])[0].astype('int8')
customers["club_member_status"] = pd.factorize(customers["club_member_status"])[0].astype('int8')
customers['postal_code'] = pd.factorize(customers['postal_code'])[0].astype('int32')
customers

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,-1612724649,-1,-1,0,0,49.0,0
1,-1740365574,-1,-1,0,0,25.0,1
2,277996312,-1,-1,0,0,24.0,2
3,-16268226,-1,-1,0,0,54.0,3
4,-1922717606,1,1,0,1,52.0,4
...,...,...,...,...,...,...,...
1371975,-1940645839,-1,-1,0,0,24.0,62927
1371976,-1245382473,-1,-1,0,0,21.0,6316
1371977,-47869340,1,1,0,1,21.0,273671
1371978,1238687594,1,1,0,1,18.0,218323


In [22]:
transactions.drop_duplicates(inplace=True)

In [23]:
# Concat the negative samples to the positive samples:
# transactions = pd.concat([transactions, chosen_neg_transactions])
transactions_merge = transactions.merge(articles, how="inner", on='article_id')
transactions_merge = transactions.merge(customers, how="inner", on='customer_id')

In [24]:
transactions_processed = transactions_merge[['t_dat','customer_id','age','article_id', 'price', 'week']].copy()

In [25]:
transactions_processed.head()

,t_dat,customer_id,age,article_id,price,week
0,2020-08-19,-732324596,34.0,784053005,0.050831,100
1,2020-08-19,-732324596,34.0,926921001,0.033881,100
2,2020-08-19,-732324596,34.0,868038001,0.050831,100
3,2020-08-19,-732324596,34.0,851124001,0.033881,100
4,2020-08-19,-732324596,34.0,897189001,0.042356,100


# Popularity based on age groups Baseline

In [26]:
bins=[20,40,60,80,100]
labels=[20, 40, 60, 80]
transactions_processed['AgeGroup'] = pd.cut(transactions_processed['age'], bins=bins, labels = labels, right=False)

In [27]:
transactions_processed.drop(
        columns=['age'], inplace=True)

In [28]:
twenties_popular = transactions_processed.loc[transactions_processed["AgeGroup"] == 20]
thirties_popular = transactions_processed.loc[transactions_processed["AgeGroup"] == 40]
fifties_popular = transactions_processed.loc[transactions_processed["AgeGroup"] == 60]
seventies_popular = transactions_processed.loc[transactions_processed["AgeGroup"] == 80]

In [29]:
twenties_popular = twenties_popular["article_id"].value_counts().rename_axis("article_id").reset_index(name="popularity")
thirties_popular = thirties_popular["article_id"].value_counts().rename_axis("article_id").reset_index(name="popularity")
fifties_popular = fifties_popular["article_id"].value_counts().rename_axis("article_id").reset_index(name="popularity")
seventies_popular = seventies_popular["article_id"].value_counts().rename_axis("article_id").reset_index(name="popularity")

In [30]:
twenties_popular['AgeGroup'] = 20
thirties_popular['AgeGroup'] = 40
fifties_popular['AgeGroup'] = 60
seventies_popular['AgeGroup'] = 80

In [31]:
# 12 most popular items for each age group
top_12_twenties = twenties_popular.sort_values("popularity",ascending=False).head(12)
top_12_thrties = twenties_popular.sort_values("popularity",ascending=False).head(12)
top_12_fourties = twenties_popular.sort_values("popularity",ascending=False).head(12)
top_12_fifties = twenties_popular.sort_values("popularity",ascending=False).head(12)

In [32]:
top_12_twenties

,article_id,popularity,AgeGroup
0,918292001,1869,20
1,915526001,1794,20
2,706016001,1518,20
3,751471001,1513,20
4,915529003,1512,20
5,898694001,1437,20
6,916468003,1352,20
7,448509014,1299,20
8,909370001,1191,20
9,714790020,1163,20


In [33]:
pop = pd.concat([twenties_popular, thirties_popular, fifties_popular, seventies_popular])

In [34]:
pop.isnull().sum()

article_id    0
popularity    0
AgeGroup      0
dtype: int64

In [35]:
transactions_processed.isnull().sum()

t_dat              0
customer_id        0
article_id         0
price              0
week               0
AgeGroup       43611
dtype: int64

In [36]:
transactions_processed = transactions_processed.dropna()
#transactions_processed['AgeGroup'] = transactions_processed['AgeGroup'].astype("Int16")

In [37]:
transactions_processed = transactions_processed.merge(pop, how="inner", on=['article_id' , 'AgeGroup'])

In [40]:
transactions_processed.sort_values(['popularity'],ascending=False)

,t_dat,customer_id,article_id,price,week,AgeGroup,popularity
214761,2020-09-08,1381822455,918292001,0.042356,102,20,1869
214062,2020-08-26,-1082099987,918292001,0.042356,101,20,1869
214064,2020-08-26,1733623814,918292001,0.042356,101,20,1869
214065,2020-08-26,-1635385455,918292001,0.042356,101,20,1869
214066,2020-09-02,-1635385455,918292001,0.042356,102,20,1869
...,...,...,...,...,...,...,...
1116287,2020-08-26,-2070344557,736511001,0.005068,101,20,1
1116286,2020-09-14,-467956315,721075001,0.023712,103,20,1
1116285,2020-08-26,-2137395417,881362003,0.050831,101,40,1
1116284,2020-08-29,1310698340,786482003,0.022017,101,40,1


In [39]:
transactions = transactions_processed

# Generating Candidates



In [41]:
# https://github.com/radekosmulski/personalized_fashion_recs/blob/main/03c_Basic_Model_Submission.ipynb
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]
transactions

,t_dat,customer_id,article_id,price,week,AgeGroup,popularity
0,2020-08-19,-732324596,784053005,0.050831,100,20,136
1,2020-08-19,-345232849,784053005,0.050831,100,20,136
2,2020-09-18,1648157038,784053005,0.050831,104,20,136
3,2020-09-12,-1732755892,784053005,0.050831,103,20,136
4,2020-08-19,-879752248,784053005,0.050831,100,20,136
...,...,...,...,...,...,...,...
1143475,2020-09-22,-881058285,903910002,0.084729,104,60,1
1143476,2020-09-22,1220833045,737994021,0.030492,104,20,1
1143477,2020-09-22,-154333942,533261032,0.033881,104,20,1
1143478,2020-09-22,-1088594216,804916001,0.016932,104,60,1


In [42]:
# https://www.kaggle.com/code/marcogorelli/radek-s-lgbmranker-starter-pack?scriptVersionId=93183348&cellId=12
last_purchased_weeks = transactions.groupby('customer_id')['week'].unique()
last_purchased_weeks_shifted = {}
for c_id, weeks in last_purchased_weeks.items():
    last_purchased_weeks_shifted[c_id] = {}
    for i in range(weeks.shape[0]-1):
        last_purchased_weeks_shifted[c_id][weeks[i]] = weeks[i+1]
    last_purchased_weeks_shifted[c_id][weeks[-1]] = test_week

In [43]:
candidates_last_purchase = transactions.copy()
weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(last_purchased_weeks_shifted[c_id][week])
    
candidates_last_purchase.week=weeks

# Candidate Bestseller

In [48]:
# https://github.com/radekosmulski/personalized_fashion_recs/blob/main/03c_Basic_Model_Submission.ipynb
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

# The combination of transactions with candidates / negative examples

In [49]:
transactions['purchased'] = 1
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)
data.purchased.mean()

0.8725289421414884

# Adding information for the bestsellers

In [50]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [51]:
data

,t_dat,customer_id,article_id,price,week,AgeGroup,popularity,purchased,bestseller_rank,product_code,...,detail_desc_12,detail_desc_13,detail_desc_14,detail_desc_15,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,2020-08-27,-2147465265,871376002,0.033881,101,40,209,1.0,999.0,-126,...,0.033200,0.050965,-0.131603,-0.001536,-1,-1,0,0,40.0,334286
1,2020-08-27,-2147465265,915529003,0.033881,101,40,819,1.0,999.0,43,...,-0.107405,-0.092542,0.092213,-0.153359,-1,-1,0,0,40.0,334286
2,2020-08-27,-2147465265,909884001,0.033881,101,40,73,1.0,999.0,97,...,-0.003656,0.013487,-0.151026,0.137418,-1,-1,0,0,40.0,334286
3,2020-08-27,-2147465265,928040002,0.067780,101,40,41,1.0,999.0,66,...,-0.054270,0.098344,-0.134060,-0.069456,-1,-1,0,0,40.0,334286
4,2020-08-27,-2147465265,874819002,0.059305,101,40,253,1.0,999.0,-70,...,0.208169,0.238553,-0.089505,-0.043328,-1,-1,0,0,40.0,334286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8414729,2020-08-19,2147478685,915529005,0.033351,105,20,369,1.0,8.0,45,...,-0.107405,-0.092542,0.092213,-0.153359,-1,-1,0,0,28.0,190365
8414730,2020-08-19,2147478685,751471001,0.033375,105,20,369,1.0,9.0,-103,...,-0.065446,-0.006451,-0.135615,-0.050629,-1,-1,0,0,28.0,190365
8414731,2020-08-19,2147478685,762846027,0.024975,105,20,369,1.0,10.0,75,...,0.110192,-0.005980,-0.028397,-0.121250,-1,-1,0,0,28.0,190365
8414732,2020-08-19,2147478685,918292001,0.041638,105,20,369,1.0,11.0,33,...,-0.134817,0.065415,0.096136,0.096682,-1,-1,0,0,28.0,190365


# Train/Test data split

In [52]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id']).copy()
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 
'bestseller_rank', 'detail_desc_0', 'detail_desc_1', 'detail_desc_3',
'detail_desc_4', 'detail_desc_5', 'detail_desc_6', 'detail_desc_7', 
'detail_desc_8', 'detail_desc_9', 'detail_desc_10', 'detail_desc_11',
'detail_desc_12', 'detail_desc_13', 'detail_desc_14', 'detail_desc_15', 
'prod_name']

In [53]:
train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

# LGBMRanker Ranking Model

In [55]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [56]:
from lightgbm.sklearn import LGBMRanker
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.786998
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.085916
[LightGBM] [Debug] init for col-wise cost 0.079743 seconds, init for row-wise cost 0.386953 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.227887 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 5231
[LightGBM] [Info] Number of data points in the train set: 4548008, number of used features: 34
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10


In [57]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.991507315327645
index_group_no 0.0019445180980224535
section_no 0.0017022182452627649
article_id 0.0006261476452081059
detail_desc_9 0.0005855999631794041
detail_desc_0 0.00045474830290357293
detail_desc_6 0.00036288537198073443
index_code 0.00034185049689986987
detail_desc_12 0.0003032550326939641
garment_group_no 0.0002843268599122026
detail_desc_10 0.0002809012434220683
age 0.00027551271202517764
detail_desc_7 0.0002461888546888036
department_no 0.00023800327380102694
detail_desc_13 0.00018957301938571852
graphical_appearance_no 0.0001895349903457063
prod_name 0.0001684941661450544
detail_desc_15 0.0001503882183761909
detail_desc_4 0.00014853817810220012
detail_desc_8 0.0
product_type_no 0.0
detail_desc_14 0.0
colour_group_code 0.0
perceived_colour_value_id 0.0
perceived_colour_master_id 0.0
detail_desc_11 0.0
detail_desc_5 0.0
detail_desc_1 0.0
detail_desc_3 0.0
FN 0.0
Active 0.0
club_member_status 0.0
fashion_news_frequency 0.0
postal_code 0.0


# Generating the predictions

In [58]:
test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

# Creating the submission file for Kaggle

In [65]:
#https://github.com/radekosmulski/personalized_fashion_recs/blob/main/01_Solution_warmup.ipynb
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

sub = pd.read_csv('C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/sample_submission.csv')
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds
sub_name = 'C:/Users/SepidehHabibiabad/OneDrive - C-Clear Partners BVBA/Desktop/AI project\AI_project_data/lgbmr_model_submission'
sub.to_csv(f'{sub_name}.csv.gz', index=False)